## Knapsack Problem

The *knapsack problem*:  given a set of integers $S = \{s_1, s_2, \ldots, s_n\}$ and a target number $T$, find a subset (i.e., knapsack) of $S$ which adds up exactly to $T$.  

For example, if $S = \{1,2,5, 9, 10\}$, there is a subset that adds up to $T = 22$, but not to $T = 23$.  Complete the following tasks related to this problem.

# 1. 

Find a subset of $S = \{1,2,5, 9, 10\}$ with sum $T = 22$.  Explain the process (algorithm) you used mentally to find the subset.  Then apply the same process in an attempt to find a subset with sum $T = 23$.  

How do you know there is no such subset?

First of all, I'm finding the highest value and start additionning with the second highest value. Then, I'm trying to add value until it isn't possible anymore. 

With this process I realize that it's not possible to arrive to 23 by additionning different values in t he subset. 


# 2.

Consider the following possible algorithm for the knapsack problem, written in psuedocode: 
```python
knapsack(S[], T):
    K = empty
    for each i < size(S)
        if sum(K) + S[i] <= T, put S[i] into K
    if sum(K) = T, return K, else return False.
```
**a)** Describe what this algorithm does in English.  

**b)** Implement this algorithm in Python and run it on the $S$ and $T$ above.


**c)** Prove that this algorithm is NOT correct.  That is, find a counterexample: a set $S$ and number $T$ for which there is a solution, but not one that the algorithm finds.

**d)** Verify that this particular $S$ and $T$ does not give the right output when entered to your Python program.

a) Basically, this algo is taking all index in a list S and if the condition is met, it appends the number into an empty list. If the sum of the new list is equal to T we return the list otherwise we return false. 


In [53]:
def knapsack_prob(S, T):
    K = []
    S = list(S)
    for i in range(len(S)):
        if sum(K) + S[i] <= T:
            K.append(S[i])
    if sum(K) == T:
        return K
    else:
        return False

S = {1,2,5,9,10}
T = 22
print(knapsack_prob(S, T))   


False


c) For example when I enter the S and T from above I end up with a return equalt to False. Why? Because we are adding one number at a time by following the the list order.

## d) Gotta find an answer

# 3. 

Another try: What if you put the elements in the knapsack from largest to smallest?  Check that this too is not a correct algorithm.

In [52]:
def knapsack_largest_to_smallest(S, T):
    K = []
    S = list(S)
    sorted_list = sorted(S)
    P = S.sort(reverse = True)
    for i in range(len(S)):
        if sum(K) + S[i] <= T:
            K.append(S[i])
    if sum(K) == T:
        return K
    else:
        return False

S = {1,2,5,9,10}
T = 22
print(knapsack_largest_to_smallest(S, T))   

[10, 9, 2, 1]


# 4.

Describe a correct algorithm for the knapsack problem (that we haven't seen in class), both in English and in pseudocode.  Then implement the algorithm in Python.  Explain how you know your algorithm is correct (even if it might not be efficient).

I found this algo called "Branch and Bound", it's an algorithm design paradigm for discccrete and combinatorial optimzation problem. It can also be used as a mathematical optimization. 

# Pseudocode 
```python
branch_and_bound
    Sort all items in decreasing order of V/ W so that upper bound 
    can be computed using Greedy Approach.
    Initialize profit, max = 0
    Create an empty queue, Q. 
    Create a dummy node of decision tree and enqueue it to Q. 
    Profit and weight of dummy node are 0. 
    Do while (Q is not empty):
        Extract an item from Q
        Compute profit of next level node. 
            If the profit is more than max:
                Update Max 
        Compute bound of next level node. 
        If bound is more than max:
            Add next level node to Q. 
    
``` 
                   

In [10]:
from collections import deque

class SimpleQueue(object):
    def __init__(self):
        self.buffer = deque()
    
    def push(self, value):
        self.buffer.appendleft(value)
    
    def pop(self):
        return self.buffer.pop()

    def __len__(self):
        return len(self.buffer)

class Node(object):
    
    def __init__(self, level, selected_items, cost, weight, bound):
        self.level = level
        self.selected_items = selected_items
        self.cost = cost 
        self.weight = weight
        self.bound = bound


def branche_and_bounds(number, capacity, weight_cost):
    """
    param number: number of existing items
    param capacity: the capacity of knapsack 
    param weight_cost: list of tuples like: [(weight, cost), (weight, cost), ...]
    return: tuple like: (best cost, best combination list (contains 1 and 0))
    """
    priority_queue = SimpleQueue()

    #sort item in non-increasing order by benfit/cost 
    ratio = [(index, item[1] / float(item[0])) for index, item in enumerate(weight_cost)]
    ratio = sorted(ratios, key=lambda x: x[1], reverse=True)

    best_so_far = Node(0, [], 0.0, 0.0, 0.0)
    a_node = Node(0, [], 0.0, 0.0, calculate_bound(best_so_far, number, capacity,
                  weight_cost, ratios))
    priority_queue.push(a_node)

    while len(priority_queue) > 0:
        curr_node = priority_queue.pop()
        if curr_node.bound > best_so_far.cost:
            curr_node_index = ratios[curr_node.level][0]
            next_item_cost = weight_cost[curr_node_index][1]
            next_item_weight = weight_cost[curr_node_index][0]
            next_added = Node(
                curr_node.level + 1,
                curr_node.selected_items + [curr_node_index],
                curr_node.cost + next_item_cost,
                curr_node.weight + next_item_weight,
                curr_node.bound
            )

            if next_added.weight <= capacity:
                if next_added.cost > best_so_far.cost:
                    best_so_far = next_added
                if next_added.bound > best_so_far.cost:
                    priority_queue.push(next_added)


            next_not_added = Node(curr_node.level + 1, curr_node.selected_items, curr_node.cost,
                                    curr_node.weight, curr_node.bound)
            next_not_added.bound = calculate_bound(next_not_added, number, capacity, weight_cost,                                                       ratios)
            if next_not_added.bound > best_so_far.cost:
                priority_queue.push(next_not_added)
    
    best_combination = [0] * number
    for wc in best_so_far.selected_items:
        best_combination[wc] = 1
    return int(best_so_far.cost), best_combination

def calculate_bound(node, number, capacity, weight_cost, ratios):
    if node.weight >= capacity:
        return 0
    else: 
        upper_bound = node.cost
        total_weight = node.weight
        current_level = node.level


        while current_level < number:
            current_index = ratios[current_level][0]

            if total_weight + weight_cost[current_index][0] > capacity:
                cost = weight_cost[current_index][1]
                weight = weight_cost[current_index][0]
                upper_bound += (capacity - total_weight) * cost/weight
                break 
            upper_bound += weight_cost[current_index][1]
            total_weight += weight_cost[current_index][0]
            current_level += 1
    
    return upper_bound




# 5. Generating correct change

Now, we will be making change using the fewest coins. 

Suppose you are a programmer for a vending machine manufacturer. Your company wants to streamline effort by giving out the fewest possible coins in change for each transaction. Suppose a customer puts in a dollar bill and purchases an item for 37 cents. What is the smallest number of coins you can use to make change? The answer is six coins: two quarters, one dime, and three pennies. 

How did we arrive at the answer of six coins? We start with the largest coin in our arsenal (a quarter) and use as many of those as possible, then we go to the next lowest coin value and use as many of those as possible. This is the greedy algorithm for change-making.

**Question:** Write the greedy algorithm for change making.

The input is the amount of change to generate (in pennies) and a list of coin sizes (in pennies)

The output is the minimum number of coins to gener

```
# buys with 1 dollar for 37 pennies
# Second argument says we can give quarters, dimes, nickels and pennies
make_change(100 - 37, [25, 10, 5, 1])

# 2 quarters, one dime, and three pennies
output --> 6 # Output would be equivalent to the choices [2, 1, 0, 3]
```

In [16]:
def greedy_pick(coins, due):
    result = []
    while (due > 0):
        print(due, coins, result)
        if (due >= coins[0]):
            num = due // coins[0]
            due -= (num * coins[0])
            result.append([coins[0], num])
        coins = coins[1:]
    return result

print(greedy_pick([25,10,5,1], 100))


100 [25, 10, 5, 1] []
[[25, 4]]


# 6 Recursive change

Write the greedy change making algorithm using recursion

In [41]:

def pickBest(coins, due):
    if due == 0: return []
    for c in coins:
        if c <= due:
            return [c] + pickBest(coins, due-c)




# 7 (Stretch) Dynamic Programming Change making

Write a solution to the change making problem using dynamic programming.

**Hint:** Start with making change for one cent and systematically work its way up to the amount of change we require. This guarantees us that at each step of the algorithm we already know the minimum number of coins needed to make change for any smaller amount. Keep a memoized table of results for each step working up to the amount of change you need to generate.